In [1]:
import pandas as pd
import numpy as np

In [2]:
ds = pd.read_csv("smsspamcollection/SMSSpamCollection", 
                 delimiter="\t", names=("class_labels", "text"))

In [3]:
ds.head()

,class_labels,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
ds.class_labels.value_counts()

ham     4825
spam     747
Name: class_labels, dtype: int64

In [5]:
X = np.array(ds.text)
y = np.array(ds.class_labels == 'spam', dtype=int)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_features = vectorizer.fit_transform(X)

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

print(cross_val_score(LogisticRegression(), X_features, y, scoring="f1", cv=10).mean())

0.932640298361


In [8]:
to_predict = ["FreeMsg: Txt: CALL to No: 86888 & claim your reward of 3 hours talk time to use from your phone now! Subscribe6GB",
              "FreeMsg: Txt: claim your reward of 3 hours talk time",
              "Have you visited the last lecture on physics?",
              "Have you visited the last lecture on physics? Just buy this book and you will have all materials! Only 99$",
              "Only 99$"]

to_predict = vectorizer.transform(to_predict)

In [9]:
cls = LogisticRegression().fit(X_features, y)
res = cls.predict(to_predict)

print(*res)

1 1 0 0 0


In [10]:
results = []
results.append(cross_val_score(LogisticRegression(), 
                               CountVectorizer(ngram_range=(2,2)).fit_transform(X), 
                               y, scoring="f1", cv=10).mean())

results.append(cross_val_score(LogisticRegression(), 
                               CountVectorizer(ngram_range=(3,3)).fit_transform(X),
                               y, scoring="f1", cv=10).mean())

results.append(cross_val_score(LogisticRegression(), 
                               CountVectorizer(ngram_range=(1,3)).fit_transform(X), 
                               y, scoring="f1", cv=10).mean())

for res in results:
    print("{0:.2f}".format(res), end=' ')

0.82 0.73 0.93 

In [11]:
from sklearn.naive_bayes import MultinomialNB

results = []
results.append(cross_val_score(MultinomialNB(), 
                               CountVectorizer(ngram_range=(2,2)).fit_transform(X), 
                               y, scoring="f1", cv=10).mean())

results.append(cross_val_score(MultinomialNB(), 
                               CountVectorizer(ngram_range=(3,3)).fit_transform(X), 
                               y, scoring="f1", cv=10).mean())
               
results.append(cross_val_score(MultinomialNB(), 
                               CountVectorizer(ngram_range=(1,3)).fit_transform(X), 
                               y, scoring="f1", cv=10).mean())
               
for res in results:
    print("{0:.2f}".format(res), end=' ')

0.65 0.38 0.89 

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

text_processing_pipeline = Pipeline([
        ('Vectorizer', TfidfVectorizer()),
        ('Classifier', LogisticRegression())
    ])

res = cross_val_score(text_processing_pipeline, X, y, scoring="f1", cv=10).mean()
print("{0:.2f}".format(res))

0.88


Качество падает

In [13]:
from sklearn.model_selection import GridSearchCV

params_grid = {'Vectorizer__ngram_range': [(1,1), (1,2), (1,3), (2,2), (2,3), (3,3)],
        'Classifier__C': [0.01, 0.1, 0.5, 1, 5, 10, 100, 200, 500, 1000, 2000]}

text_processing_pipeline = Pipeline([
        ('Vectorizer', CountVectorizer()),
        ('Classifier', LogisticRegression())
    ])

res = GridSearchCV(text_processing_pipeline, params_grid, scoring='f1', cv=10, n_jobs=-1).fit(X, y)

print(res.best_score_)
print(res.best_params_)

0.940425995066
{'Classifier__C': 1000, 'Vectorizer__ngram_range': (1, 1)}


Вывод: модель с униграммами иногда оказывается лучше более сложных моделей.

In [14]:
params_grid = {'Vectorizer__ngram_range': [(1,1), (1,2), (1,3), (2,2), (2,3), (3,3)],
               'Classifier__alpha': [0, 0.1, 0.2, 0.5, 1, 2, 5, 10]}

text_processing_pipeline = Pipeline([
        ('Vectorizer', CountVectorizer()),
        ('Classifier', MultinomialNB())
    ])

res = GridSearchCV(text_processing_pipeline, params_grid, scoring='f1', cv=10).fit(X, y)

print(res.best_score_)
print(res.best_params_)

0.955682590573
{'Vectorizer__ngram_range': (1, 3), 'Classifier__alpha': 0.2}
